# Import Required Libraries
Import the necessary libraries, including the OpenAI library.

In [1]:
import os
import json
import openai
import random

# Set API Key
Set the OpenAI API key using an environment variable or directly in the code.

In [2]:
#set the openai api key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Test Connection
# Uncomment the following code to test the connection


#response = openai.chat.completions.create(
#    model="gpt-4o-mini",
#    messages=[{"role": "user", "content": "Say this is a test"}],
#    stream=True,
#)

#print(response, "\n")

# Define Enterprise System Functions
In this cell, we define the functions that will be used to fetch real-time data from the Enterprise's systems. These functions will simulate the retrieval of shield percentage and core temperature.

In [3]:
#
enterprise_system_functions = [
        { 
            "type": "function",
            "function": {
                "name": "get_shield_percentage",
                "description": "Get the current shield percentage",

            },
        },
        
        { 
            "type": "function",
            "function": {
                "name": "get_core_temperature",
                "description": "Get the internal temperature of the dylityum core",

            }
        }
    ]

# Implement functions
These are the functions that will be called based on the description above. The neme of the function must match the name in the function description

In [4]:
#The functions implementations. This is where the actual function is defined. This is the function that will be called to get the actual data.

def get_shield_percentage():
    # Simulate changing shield status
    shield_status = {
        "shiled_percentage": 68 - random.randint(5, 15)
    }
    return json.dumps(shield_status)

def get_core_temperature():
    # Simulate core temperature
    core_temperature = {
        "dilithium_core_temp": 4500 
    }
        
    return json.dumps(core_temperature)

# Calls to OPENAI
- ask_openai function will return the answer to user prompt like CHAT GPT, without any knowledge about the internal systems
- ask_openai_with_functions function will pass the information about the available functions. In case a proper function is found my matching the desctiption, the function name and parameters will be included in the response
- ask_openai_complete_prompt will have a parameter that is the result of calling the internal function that the API returned in the previus call


In [5]:

# OpenAI API call
# This function sends a prompt to the OpenAI API and returns the response. This is the request that does not use the tooling feature.
def ask_openai(prompt: str):
    try:
        print(f"An error occurred: {e}")
        default_response = openai.chat.completions.create(
            model="gpt-4o-mini",  
            messages=[{"role": "system", "content": "You are an AI assistant aboard the starship Enterprise."},
                    {"role": "user", "content": prompt}],
            max_tokens=150,
            )
        return default_response
    except openai.error.OpenAIError as e:
        print(f"An error occurred: {e}")
# OpenAI API call
# This function sends a prompt to the OpenAI API and returns the response. This is the request that passes the tooling feature.This will return the function name and the parameters that will be called.
def ask_openai_with_functions(prompt: str):
    try:
        initial_function_response = openai.chat.completions.create(
            model="gpt-4o-mini",  
            messages=[{"role": "system", "content":
                        "You are an AI assistant aboard the starship Enterprise."},
                      {"role": "user", "content": prompt}],
            tools=enterprise_system_functions,
            max_tokens=150,
            tool_choice="auto"
        )
        return initial_function_response
    except openai.error.OpenAIError as e:
        print(f"An error occurred: {e}")

# OpenAI API call
# This function sends a prompt to the OpenAI API and returns the response. This is the request that passes the result of the function call to the AI. This will return the final response containing the data provided by the tools.
def ask_openai_complete_prompt(prompt: str, output, content):
    try:
        final_response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "user", "content": prompt},
                {"role": "function", "name": output.name, "content": content},
            ],
        )
        return final_response
    except openai.error.OpenAIError as e:
        print(f"An error occurred: {e}")


# Run the scenario
The fictional scenario will:
- call the Open AI API without any function calling
- call the API providing function information and get function name and parameters from API response
- call the API with the same prompt and the infomation retrieved from the local function.

In [6]:





print("[SCENARIO START]")
print("Enterprise is under attack. The Captain queries the AI for status:\n")

# Captain asks the AI for shield status and core temperature
query1 = "What is the current shield percentage?"
query2 = "What is the dilithium core temperature?"

#Call the OpenAI API with no tooling
print(f"Captain: {query1}")
response1 = ask_openai(query1)
print(f"AI: {response1.choices[0].message.content}")

print(f"\nCaptain: {query2}")
response2 = ask_openai(query2)
print(f"AI: {response2.choices[0].message.content}")

# Captain realizes the data is incorrect and asks for a solution
print("\n[ALERT] First Officer: 'This data doesn't help the situation!'")
print("Captain: 'AI, how can we get accurate data?'")

solution_query = "How can you fetch real-time data?"
solution_response = ask_openai(solution_query)
print(f"AI: {solution_response.choices[0].message.content}")

# Implement tooling
print("\n[TOOL ENABLED] Connecting AI to the Enterprise telemetry system...\n")

# Captain re-asks the questions
print(f"Captain: {query1}")

# the call to the function that has te information of the tools available.
# This example uses 2 calls to get the infomration for the shields and temperature, but it can be implemented to use multiple functions on a single API call. 
#This should be done to avoid multiple calls to the API, but is done like this for easy reading and simplicity.
live_response1 = ask_openai_with_functions(query1)

#gets the function that will be called
if live_response1.choices[0].finish_reason=="tool_calls":
    function = live_response1.choices[0].message.tool_calls[0].function
    print(f"AI: I will connect to the systems and get the info by calling  {function.name}")

#call the local function to get the data
eval_function = eval(function.name)
content = eval_function()

#send back the prompt with the function and the data and get the final response
final_response_1  = ask_openai_complete_prompt(query1, function, content)


print(final_response_1.choices[0].message.content)

print(f"Captain: {query2}")
live_response2 = ask_openai_with_functions(query2)


if live_response2.choices[0].finish_reason=="tool_calls":
    function = live_response2.choices[0].message.tool_calls[0].function
    print(f"AI: I will connect to the systems and get the info by calling  {function.name}")

eval_function = eval(function.name)
content = eval_function()

final_response_2  = ask_openai_complete_prompt(query2, function, content)


print(final_response_2.choices[0].message.content)


# Victory
print("\n[SCENARIO END]")
print("Captain: 'Accurate data saved the day! Reroute power to shields and vent heat from the core!'")


[SCENARIO START]
Enterprise is under attack. The Captain queries the AI for status:

Captain: What is the current shield percentage?
AI: As an AI assistant, I am unable to access real-time systems or current readings aboard the Enterprise. Please check the ship's tactical display or consult a bridge officer for the latest shield status.

Captain: What is the dilithium core temperature?
AI: The dilithium core temperature can vary depending on the specific conditions of the warp core in a starship, such as the USS Enterprise. In general, dilithium crystals are used in conjunction with matter-antimatter reactions to regulate and control the warp field. While the exact temperature is not typically specified in Star Trek canon, it is essential for the dilithium crystals to be maintained at optimal conditions to prevent destabilization and ensure efficient warp drive operation.

If you're looking for a specific figure in degrees Kelvin or Celsius, the series does not provide that level of te